# Verteilung der Nutztiere in der Schweiz

## Einlesen der Daten:

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import numpy as np

### Daten zu Tieren pro Gemeinde/Bezirk/Kanton:

In [2]:
tiere = pd.read_excel("Rohdaten/TiereEntwicklung.xlsx") #Daten von 1997, 2007 und 2017

In [3]:
tiere.head(5)

,Art,ID,Ort,1997,2007,2017
0,Betriebe,1,Schweiz,77730,61764,51620
1,Betriebe,2,- Zürich,5024,4155,3432
2,Betriebe,3,>> Bezirk Affoltern,399,329,275
3,Betriebe,4,......0001 Aeugst am Albis,30,18,14
4,Betriebe,5,......0002 Affoltern am Albis,33,28,22


In [4]:
# beim mergen stellt sich später ein Problem mit Horgen raus (neue Nummer durch Gemeindefusion, ich prüfe hier)
tiere[(tiere["Ort"] == "......0295 Horgen")]

,Art,ID,Ort,1997,2007,2017
111,Betriebe,112,......0295 Horgen,83,74,61
2503,Rinder,112,......0295 Horgen,2505,2316,2320
4895,Pferde,112,......0295 Horgen,146,150,148
7287,Schafe,112,......0295 Horgen,254,137,153
9679,Ziegen,112,......0295 Horgen,42,49,31
12071,Schweine,112,......0295 Horgen,676,543,856
14463,Geflügel,112,......0295 Horgen,1426,989,809
16855,Übrige,112,......0295 Horgen,297,182,85


### Daten zur Bevölkerung pro Gemeinde/Bezirk/Kanton, inkl. Bereinigung

In [13]:
bevoelkerung = pd.read_excel("Rohdaten/BevölkerungEntwicklung.xlsx") #Daten von 1997, 2007 und 2017
bevoelkerung.head(5)

,Bevölkerung Jahr,Ort,Anzahl
0,1997,Schweiz,7081346
1,1997,- Zürich,1178848
2,1997,>> Bezirk Affoltern,39263
3,1997,......0001 Aeugst am Albis,1431
4,1997,......0002 Affoltern am Albis,9451


In [14]:
bevoelkerung = bevoelkerung.pivot(index='Ort', columns='Bevölkerung Jahr', values='Anzahl') #brauchbar anordnen

In [15]:
bevoelkerung.head(5)

Bevölkerung Jahr,1997,2007,2017
Ort,,,
- Aargau,531665,574813,663462
- Appenzell Ausserrhoden,54136,52509,54954
- Appenzell Innerrhoden,14833,15300,16003
- Basel-Landschaft,253911,267166,285624
- Basel-Stadt,194913,184822,193070


In [16]:
bevoelkerung = bevoelkerung.reset_index()

In [17]:
bevoelkerung.columns.name = None

In [19]:
#Spalten umbenennen
spalten = {
    1997: 'Einwohner 1997',
    2007: 'Einwohner 2007',
    2017: 'Einwohner 2017'
}

In [20]:
bevoelkerung = bevoelkerung.rename(columns=spalten)

In [21]:
bevoelkerung.head(5)

,Ort,Einwohner 1997,Einwohner 2007,Einwohner 2017
0,- Aargau,531665,574813,663462
1,- Appenzell Ausserrhoden,54136,52509,54954
2,- Appenzell Innerrhoden,14833,15300,16003
3,- Basel-Landschaft,253911,267166,285624
4,- Basel-Stadt,194913,184822,193070


In [22]:
# auch hier der Horgen-Test
bevoelkerung[(bevoelkerung["Ort"] == "......0133 Horgen")]

,Ort,Einwohner 1997,Einwohner 2007,Einwohner 2017
121,......0133 Horgen,16535,18074,20291


### Die beiden Tabellen mergen und sinnvoller beschriften:

In [23]:
alle = tiere.merge(bevoelkerung, on='Ort', how='left')
alle.head(5)

,Art,ID,Ort,1997,2007,2017,Einwohner 1997,Einwohner 2007,Einwohner 2017
0,Betriebe,1,Schweiz,77730,61764,51620,7081346.0,7508739.0,8419550.0
1,Betriebe,2,- Zürich,5024,4155,3432,1178848.0,1284052.0,1487969.0
2,Betriebe,3,>> Bezirk Affoltern,399,329,275,39263.0,44635.0,52904.0
3,Betriebe,4,......0001 Aeugst am Albis,30,18,14,1431.0,1645.0,1977.0
4,Betriebe,5,......0002 Affoltern am Albis,33,28,22,9451.0,10302.0,11900.0


In [24]:
#Welche Orte haben keine Einwohner erhalten. Hier u.a. Horgen (aufgrund von diesem Test checke ich oben nochmals)
alle[(alle["Einwohner 2017"].isnull()) & (alle["Art"] == "Betriebe")]

,Art,ID,Ort,1997,2007,2017,Einwohner 1997,Einwohner 2007,Einwohner 2017
111,Betriebe,112,......0295 Horgen,83,74,61,NaN,NaN,NaN
164,Betriebe,165,......0294 Elgg,54,47,37,NaN,NaN,NaN
1146,Betriebe,1147,>> Kanton Appenzell Innerrhoden,698,567,465,NaN,NaN,NaN
1246,Betriebe,1247,......3544 Bergün Filisur,23,20,18,NaN,NaN,NaN
1250,Betriebe,1251,>> Region Engiadina Bassa / Val Müstair,319,258,215,NaN,NaN,NaN
1306,Betriebe,1307,>> Region Prättigau / Davos,694,521,414,NaN,NaN,NaN
2254,Betriebe,2255,>> Canton de Neuchâtel,1288,976,807,NaN,NaN,NaN
2261,Betriebe,2262,......6417 La Grande-Béroche,100,80,63,NaN,NaN,NaN


In [25]:
alle.rename(columns={"1997": "Tiere 1997", "2007": "Tiere 2007", "2017": "Tiere 2017"}, inplace=True)

In [26]:
alle.head(5)

,Art,ID,Ort,Tiere 1997,Tiere 2007,Tiere 2017,Einwohner 1997,Einwohner 2007,Einwohner 2017
0,Betriebe,1,Schweiz,77730,61764,51620,7081346.0,7508739.0,8419550.0
1,Betriebe,2,- Zürich,5024,4155,3432,1178848.0,1284052.0,1487969.0
2,Betriebe,3,>> Bezirk Affoltern,399,329,275,39263.0,44635.0,52904.0
3,Betriebe,4,......0001 Aeugst am Albis,30,18,14,1431.0,1645.0,1977.0
4,Betriebe,5,......0002 Affoltern am Albis,33,28,22,9451.0,10302.0,11900.0


### File speichern als .csv:

In [27]:
alle.to_csv('Rohdaten/einwohnerfehlen.csv')

### mit kompletten Einwohnerdaten wieder hochladen:

In [28]:
# Da es wenige Orte ohne Einwohner waren (zb Horgen), erfasse ich dich kurz im Excel und lade wieder hoch
alle = pd.read_csv("Rohdaten/nutztieremitalleneinwohnern.csv")

In [29]:
# jetzt stimmts, alle haben Einwohner
alle[(alle["Einwohner 2017"].isnull())]

,Unnamed: 0,Art,ID,Ort,Tiere 1997,Tiere 2007,Tiere 2017,Einwohner 1997,Einwohner 2007,Einwohner 2017


### Spalten sinnvoller anordnen: 

In [30]:
alle = alle[["ID", "Ort", "Art", "Tiere 1997", "Tiere 2007", "Tiere 2017", "Einwohner 1997", "Einwohner 2007", "Einwohner 2017"]]

In [31]:
alle.head(5)

,ID,Ort,Art,Tiere 1997,Tiere 2007,Tiere 2017,Einwohner 1997,Einwohner 2007,Einwohner 2017
0,1,Schweiz,Betriebe,77730,61764,51620,7081346,7508739,8419550
1,2,- Zürich,Betriebe,5024,4155,3432,1178848,1284052,1487969
2,3,>> Bezirk Affoltern,Betriebe,399,329,275,39263,44635,52904
3,4,......0001 Aeugst am Albis,Betriebe,30,18,14,1431,1645,1977
4,5,......0002 Affoltern am Albis,Betriebe,33,28,22,9451,10302,11900


### Namen bereinigen, Einheit für Orte zuteilen:

In [32]:
alle["Ort"]

0                              Schweiz
1                             - Zürich
2                  >> Bezirk Affoltern
3           ......0001 Aeugst am Albis
4        ......0002 Affoltern am Albis
5                ......0003 Bonstetten
6           ......0004 Hausen am Albis
7                  ......0005 Hedingen
8           ......0006 Kappel am Albis
9                    ......0007 Knonau
10              ......0008 Maschwanden
11           ......0009 Mettmenstetten
12                 ......0010 Obfelden
13                ......0011 Ottenbach
14               ......0012 Rifferswil
15                ......0013 Stallikon
16        ......0014 Wettswil am Albis
17               >> Bezirk Andelfingen
18                  ......0021 Adlikon
19              ......0022 Benken (ZH)
20           ......0023 Berg am Irchel
21           ......0024 Buch am Irchel
22                  ......0025 Dachsen
23                     ......0026 Dorf
24              ......0027 Feuerthalen
25                   ....

In [33]:
# Test für Gemeinde
re.search(r"^\.{6}\d{4} .+$", "......0001 Aeugst am Albis")

<re.Match object; span=(0, 26), match='......0001 Aeugst am Albis'>

In [34]:
# Test für Bezirke
re.search(r"^>> .+$", ">> Wahlkreis Luzern-Stadt")

<re.Match object; span=(0, 25), match='>> Wahlkreis Luzern-Stadt'>

In [35]:
# Test für Kantone
re.search(r"^- .+$", "- Aargau")

<re.Match object; span=(0, 8), match='- Aargau'>

In [36]:
# Test für Land
re.search(r"Schweiz", "Schweiz")

<re.Match object; span=(0, 7), match='Schweiz'>

### Testen, ob Gemeinden auch als Gemeinden erkannt werden:

In [37]:
alle["Ort"].str.contains(r"^\.{6}\d{4} .+$")

0        False
1        False
2        False
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17       False
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
         ...  
19106     True
19107     True
19108     True
19109     True
19110     True
19111     True
19112     True
19113     True
19114    False
19115     True
19116     True
19117     True
19118     True
19119     True
19120     True
19121     True
19122     True
19123     True
19124     True
19125     True
19126     True
19127     True
19128     True
19129     True
19130     True
19131     True
19132     True
19133     True
19134     True
19135     True
Name: Ort, Length: 19136, dtype: bool

### Neue Spalte mit Einheit für Gemeinden, Bezirke, Kantone, Land: 

In [38]:
alle.loc[alle['Ort'].str.contains(r"^\.{6}\d{4} .+$"), 'Einheitstyp'] = "Gemeinde"
alle.loc[alle['Ort'].str.contains(r"^>> .+$"), 'Einheitstyp'] = "Bezirk"
alle.loc[alle['Ort'].str.contains(r"^- .+$"), 'Einheitstyp'] = "Kanton"
alle.loc[alle['Ort'].str.contains(r"Schweiz"), 'Einheitstyp'] = "Land"

In [39]:
alle

,ID,Ort,Art,Tiere 1997,Tiere 2007,Tiere 2017,Einwohner 1997,Einwohner 2007,Einwohner 2017,Einheitstyp
0,1,Schweiz,Betriebe,77730,61764,51620,7081346,7508739,8419550,Land
1,2,- Zürich,Betriebe,5024,4155,3432,1178848,1284052,1487969,Kanton
2,3,>> Bezirk Affoltern,Betriebe,399,329,275,39263,44635,52904,Bezirk
3,4,......0001 Aeugst am Albis,Betriebe,30,18,14,1431,1645,1977,Gemeinde
4,5,......0002 Affoltern am Albis,Betriebe,33,28,22,9451,10302,11900,Gemeinde
5,6,......0003 Bonstetten,Betriebe,22,19,14,3561,4612,5435,Gemeinde
6,7,......0004 Hausen am Albis,Betriebe,51,45,35,3034,3256,3571,Gemeinde
7,8,......0005 Hedingen,Betriebe,17,13,11,2802,3307,3687,Gemeinde
8,9,......0006 Kappel am Albis,Betriebe,27,26,22,855,857,1110,Gemeinde
9,10,......0007 Knonau,Betriebe,27,27,25,1228,1583,2168,Gemeinde


### Einheitsnummer der Gemeinden beibehalten in neuer Spalte:

In [40]:
alle['Ort'].str.extract(r"^\.{6}(\d{4}) .+$")

,0
0,NaN
1,NaN
2,NaN
3,0001
4,0002
5,0003
6,0004
7,0005
8,0006
9,0007


In [41]:
alle['Ortsnummer'] = alle['Ort'].str.extract(r"^\.{6}(\d{4}) .+$")
alle.head(30)

,ID,Ort,Art,Tiere 1997,Tiere 2007,Tiere 2017,Einwohner 1997,Einwohner 2007,Einwohner 2017,Einheitstyp,Ortsnummer
0,1,Schweiz,Betriebe,77730,61764,51620,7081346,7508739,8419550,Land,NaN
1,2,- Zürich,Betriebe,5024,4155,3432,1178848,1284052,1487969,Kanton,NaN
2,3,>> Bezirk Affoltern,Betriebe,399,329,275,39263,44635,52904,Bezirk,NaN
3,4,......0001 Aeugst am Albis,Betriebe,30,18,14,1431,1645,1977,Gemeinde,0001
4,5,......0002 Affoltern am Albis,Betriebe,33,28,22,9451,10302,11900,Gemeinde,0002
5,6,......0003 Bonstetten,Betriebe,22,19,14,3561,4612,5435,Gemeinde,0003
6,7,......0004 Hausen am Albis,Betriebe,51,45,35,3034,3256,3571,Gemeinde,0004
7,8,......0005 Hedingen,Betriebe,17,13,11,2802,3307,3687,Gemeinde,0005
8,9,......0006 Kappel am Albis,Betriebe,27,26,22,855,857,1110,Gemeinde,0006
9,10,......0007 Knonau,Betriebe,27,27,25,1228,1583,2168,Gemeinde,0007


### Ortsname bestimmen/anpassen:

In [42]:
alle['Ortsname'] = alle['Ort']

In [43]:
#Gemeinden
alle["Ortsname"] = alle['Ortsname'].str.replace(r"^\.{6}\d{4} ", "")

In [44]:
#Bezirke
alle["Ortsname"] = alle['Ortsname'].str.replace(r"^>> ", "")

In [45]:
#Kantone
alle["Ortsname"] = alle['Ortsname'].str.replace(r"^- ", "")

In [46]:
alle.head(5)

,ID,Ort,Art,Tiere 1997,Tiere 2007,Tiere 2017,Einwohner 1997,Einwohner 2007,Einwohner 2017,Einheitstyp,Ortsnummer,Ortsname
0,1,Schweiz,Betriebe,77730,61764,51620,7081346,7508739,8419550,Land,NaN,Schweiz
1,2,- Zürich,Betriebe,5024,4155,3432,1178848,1284052,1487969,Kanton,NaN,Zürich
2,3,>> Bezirk Affoltern,Betriebe,399,329,275,39263,44635,52904,Bezirk,NaN,Bezirk Affoltern
3,4,......0001 Aeugst am Albis,Betriebe,30,18,14,1431,1645,1977,Gemeinde,0001,Aeugst am Albis
4,5,......0002 Affoltern am Albis,Betriebe,33,28,22,9451,10302,11900,Gemeinde,0002,Affoltern am Albis


### Nur noch Angaben, die ich brauche anzeigen:


In [47]:
alle = alle[["Ortsname", "Einheitstyp", "Art", "Tiere 1997", "Tiere 2007", "Tiere 2017", "Einwohner 1997", "Einwohner 2007", "Einwohner 2017", "Ortsnummer"]]

In [48]:
alle.head(5)

,Ortsname,Einheitstyp,Art,Tiere 1997,Tiere 2007,Tiere 2017,Einwohner 1997,Einwohner 2007,Einwohner 2017,Ortsnummer
0,Schweiz,Land,Betriebe,77730,61764,51620,7081346,7508739,8419550,NaN
1,Zürich,Kanton,Betriebe,5024,4155,3432,1178848,1284052,1487969,NaN
2,Bezirk Affoltern,Bezirk,Betriebe,399,329,275,39263,44635,52904,NaN
3,Aeugst am Albis,Gemeinde,Betriebe,30,18,14,1431,1645,1977,0001
4,Affoltern am Albis,Gemeinde,Betriebe,33,28,22,9451,10302,11900,0002


## File für Berechnungen vorbereiten:

### Anteil Tiere/Betriebe pro Einwohner:

In [49]:
alle['Anteil 2017'] = (alle['Einwohner 2017'] / alle['Tiere 2017']).round(2)

/Users/Fox/.virtualenvs/Hallo/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
alle['Anteil 2007'] = (alle['Einwohner 2007'] / alle['Tiere 2007']).round(2)

In [51]:
alle['Anteil 1997'] = (alle['Einwohner 1997'] / alle['Tiere 1997']).round(2)

In [52]:
alle

,Ortsname,Einheitstyp,Art,Tiere 1997,Tiere 2007,Tiere 2017,Einwohner 1997,Einwohner 2007,Einwohner 2017,Ortsnummer,Anteil 2017,Anteil 2007,Anteil 1997
0,Schweiz,Land,Betriebe,77730,61764,51620,7081346,7508739,8419550,NaN,163.110000,121.57,91.100000
1,Zürich,Kanton,Betriebe,5024,4155,3432,1178848,1284052,1487969,NaN,433.560000,309.04,234.640000
2,Bezirk Affoltern,Bezirk,Betriebe,399,329,275,39263,44635,52904,NaN,192.380000,135.67,98.400000
3,Aeugst am Albis,Gemeinde,Betriebe,30,18,14,1431,1645,1977,0001,141.210000,91.39,47.700000
4,Affoltern am Albis,Gemeinde,Betriebe,33,28,22,9451,10302,11900,0002,540.910000,367.93,286.390000
5,Bonstetten,Gemeinde,Betriebe,22,19,14,3561,4612,5435,0003,388.210000,242.74,161.860000
6,Hausen am Albis,Gemeinde,Betriebe,51,45,35,3034,3256,3571,0004,102.030000,72.36,59.490000
7,Hedingen,Gemeinde,Betriebe,17,13,11,2802,3307,3687,0005,335.180000,254.38,164.820000
8,Kappel am Albis,Gemeinde,Betriebe,27,26,22,855,857,1110,0006,50.450000,32.96,31.670000
9,Knonau,Gemeinde,Betriebe,27,27,25,1228,1583,2168,0007,86.720000,58.63,45.480000


### File speichern als .csv:

In [53]:
#Sammlung und Bereinigung ist beendet. Ich speichere, um im Notebook "Nutztiere Auswertung" direkt starten zu können:
alle.to_csv('BereinigteDaten/nutztiereschweizfertig.csv')